In [172]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load MNIST dataset

In [173]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [174]:
alpha = 5

numTrainingSamples = mnist.train.images.shape[0] # 55000
numFeatures = mnist.train.images.shape[1] # 784
numClasses = mnist.train.labels.shape[1] # 10
# numHiddenUnits = numTrainingSamples/(alpha * (numFeatures + numClasses))
numHiddenUnits = 200
numEpochs = 10000
batchSize = 200

### One-layer Neural Network

In [175]:
X = tf.placeholder(tf.float32, shape=[None, numFeatures])
y = tf.placeholder(tf.float32, shape=[None, numClasses])

In [176]:
W1 = tf.Variable(tf.truncated_normal([numFeatures, numHiddenUnits], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1), [numHiddenUnits])

W2 = tf.Variable(tf.truncated_normal([numHiddenUnits, numClasses], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1), [numClasses])

In [177]:
hiddenLayerOuput = tf.nn.relu(tf.matmul(X, W1) + B1)
finalOutput = tf.nn.relu(tf.matmul(hiddenLayerOuput, W2) + B2)

In [178]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=y,
        logits=finalOutput
    )
)

optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [179]:
correct_prediction = tf.equal(tf.argmax(finalOutput, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [180]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(numEpochs):
    batch = mnist.train.next_batch(batch_size=batchSize)
    sess.run(optimizer, feed_dict={X: batch[0], y: batch[1]})
    if ((i+1)<20 or (i+1)%1000 == 0):
        batch_accuracy = accuracy.eval(feed_dict={X: batch[0], y: batch[1]})
        print("Iteration %s, training accuracy: %s" % ((i+1), batch_accuracy))

Iteration 1, training accuracy: 0.22
Iteration 2, training accuracy: 0.295
Iteration 3, training accuracy: 0.29
Iteration 4, training accuracy: 0.325
Iteration 5, training accuracy: 0.31
Iteration 6, training accuracy: 0.35
Iteration 7, training accuracy: 0.395
Iteration 8, training accuracy: 0.415
Iteration 9, training accuracy: 0.4
Iteration 10, training accuracy: 0.47
Iteration 11, training accuracy: 0.46
Iteration 12, training accuracy: 0.46
Iteration 13, training accuracy: 0.45
Iteration 14, training accuracy: 0.555
Iteration 15, training accuracy: 0.55
Iteration 16, training accuracy: 0.595
Iteration 17, training accuracy: 0.575
Iteration 18, training accuracy: 0.565
Iteration 19, training accuracy: 0.625
Iteration 1000, training accuracy: 0.84
Iteration 2000, training accuracy: 0.85
Iteration 3000, training accuracy: 0.965
Iteration 4000, training accuracy: 0.95
Iteration 5000, training accuracy: 0.98
Iteration 6000, training accuracy: 0.96
Iteration 7000, training accuracy: 0.9

In [181]:
training_accuracy = accuracy.eval(feed_dict={X: mnist.train.images, y: mnist.train.labels}) * 100
testing_accuracy = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels}) * 100

predictions = sess.run(tf.argmax(tf.nn.softmax(finalOutput), 1), feed_dict={X: mnist.test.images})
labels = sess.run(tf.argmax(mnist.test.labels, 1))

print("==========================================================")
print("===== One-layer Neural Network with %s hidden units =====" % numHiddenUnits)
print("==========================================================")
print("Training accuracy:\t %.3f %%" % training_accuracy)
print("Testing accuracy:\t %.3f %%" % testing_accuracy)

print("\nConfusion Matrix: \n")
print(sess.run(tf.confusion_matrix(labels=labels, predictions=predictions)))

===== One-layer Neural Network with 200 hidden units =====
Training accuracy:	 98.275 %
Testing accuracy:	 97.050 %

Confusion Matrix: 

[[ 911    1   13    1    2   11   19   11    3    8]
 [   0 1124    3    1    0    1    3    0    3    0]
 [   1    3 1002    7    4    0    2    7    6    0]
 [   0    0    6  986    0    7    0    4    4    3]
 [   1    0    4    1  962    0    2    2    1    9]
 [   2    1    0    5    1  873    5    0    3    2]
 [   2    4    2    1    6    8  932    0    3    0]
 [   0    4   12    2    0    0    0 1000    3    7]
 [   1    1    5    6    6    4    6    4  940    1]
 [   2    6    1    6    6    6    0    5    2  975]]


### Two-layter Neural Networks

In [201]:
alpha = 5

numTrainingSamples = mnist.train.images.shape[0] # 55000
numFeatures = mnist.train.images.shape[1] # 784
numClasses = mnist.train.labels.shape[1] # 10
# alpha = 5
# numHiddenUnits = numTrainingSamples/(alpha * (numFeatures + numClasses))
numHiddenUnits1 = 200
numHiddenUnits2 = 200
numEpochs = 10000
batchSize = 200

In [202]:
X = tf.placeholder(tf.float32, shape=[None, numFeatures])
y = tf.placeholder(tf.float32, shape=[None, numClasses])

In [203]:
W1 = tf.Variable(tf.truncated_normal([numFeatures, numHiddenUnits1], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1), [numHiddenUnits1])

W2 = tf.Variable(tf.truncated_normal([numHiddenUnits1, numHiddenUnits2], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1), [numHiddenUnits2])

W3 = tf.Variable(tf.truncated_normal([numHiddenUnits2, numClasses], stddev=0.1))
B3 = tf.Variable(tf.constant(0.1), [numClasses])

In [204]:
hiddenLayer1Output = tf.nn.relu(tf.matmul(X, W1) + B1)
hiddenLayer2Output = tf.nn.relu(tf.matmul(hiddenLayer1Output, W2) + B2)
finalOutput = tf.nn.relu(tf.matmul(hiddenLayer2Output, W3) + B3)

In [207]:
correct_prediction = tf.equal(tf.argmax(finalOutput,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [208]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=y,
        logits=finalOutput
    )
)

optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [209]:
sess.run(tf.global_variables_initializer())

for i in range(numEpochs):
    batch = mnist.train.next_batch(batch_size=batchSize)
    sess.run(optimizer, feed_dict={X: batch[0], y: batch[1]})
    if ((i+1)<20 or (i+1)%1000 == 0):
        batch_accuracy = accuracy.eval(feed_dict={X: batch[0], y: batch[1]})
        print("Iteration %s, training accuracy: %s" % ((i+1), batch_accuracy))

Iteration 1, training accuracy: 0.28
Iteration 2, training accuracy: 0.23
Iteration 3, training accuracy: 0.285
Iteration 4, training accuracy: 0.34
Iteration 5, training accuracy: 0.385
Iteration 6, training accuracy: 0.365
Iteration 7, training accuracy: 0.425
Iteration 8, training accuracy: 0.46
Iteration 9, training accuracy: 0.45
Iteration 10, training accuracy: 0.46
Iteration 11, training accuracy: 0.52
Iteration 12, training accuracy: 0.64
Iteration 13, training accuracy: 0.565
Iteration 14, training accuracy: 0.55
Iteration 15, training accuracy: 0.6
Iteration 16, training accuracy: 0.59
Iteration 17, training accuracy: 0.685
Iteration 18, training accuracy: 0.655
Iteration 19, training accuracy: 0.745
Iteration 1000, training accuracy: 0.99
Iteration 2000, training accuracy: 0.98
Iteration 3000, training accuracy: 1.0
Iteration 4000, training accuracy: 0.99
Iteration 5000, training accuracy: 1.0
Iteration 6000, training accuracy: 0.995
Iteration 7000, training accuracy: 1.0
It

In [213]:
training_accuracy = accuracy.eval(feed_dict={X: mnist.train.images, y: mnist.train.labels}) * 100
testing_accuracy = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels}) * 100

predictions = sess.run(tf.argmax(tf.nn.softmax(finalOutput), 1), feed_dict={X: mnist.test.images})
labels = sess.run(tf.argmax(mnist.test.labels, 1))

print("=================================================================")
print("===== Two-layer Neural Network with (%s, %s) hidden units =====" % (numHiddenUnits1, numHiddenUnits2))
print("=================================================================")
print("Training accuracy:\t %.3f %%" % training_accuracy)
print("Testing accuracy:\t %.3f %%" % testing_accuracy)

print("\nConfusion Matrix: \n")
print(sess.run(tf.confusion_matrix(labels=labels, predictions=predictions)))

===== Two-layer Neural Network with (200, 200) hidden units =====
Training accuracy:	 99.918 %
Testing accuracy:	 97.670 %

Confusion Matrix: 

[[ 969    0    1    0    0    2    2    2    2    2]
 [   0 1123    2    3    0    0    2    1    4    0]
 [   4    1 1000    5    2    0    3   10    7    0]
 [   0    0    7  990    0    2    0    3    3    5]
 [   2    0    3    2  955    0    2    4    1   13]
 [   3    0    1   10    1  866    4    0    5    2]
 [   6    2    0    1    2    6  935    1    5    0]
 [   0    1    5    4    1    0    0 1009    4    4]
 [   4    0    3    8    4    2    5    4  942    2]
 [   1    3    0    4    8    2    2    7    4  978]]


### Reference

adeshpande3 - <a href="https://github.com/adeshpande3/Tensorflow-Programs-and-Tutorials/blob/master/Simple%20Neural%20Networks.ipynb">Tensorflow-Programs-and-Tutorials</a>